In [ ]:
#!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# 19 April 2023: 
# Disappointing update, trained on kaggle forum posts in the hopes that SO would be filled out (into significant other)
# It definitely did worse. They've definitely cleaned NSFW from the dataset also. Alas
# Really should expand to trigrams, but... time... 

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# 09 April 2023
# nrobot
# Run the acornym expansion on the full dataset!

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pandas as pd

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import gensim
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_short, remove_stopwords, strip_multiple_whitespaces
from nltk.corpus import stopwords

import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models import Word2Vec

from nltk import PorterStemmer

In [ ]:
columns_page_info = ['time_downloaded', 'author', 'posted_date_readable',  'post_ordinal', 'thread_page_name', 'thread_page_num', 'thread_page_url', 'post_text']

columns_thread_info = ['src_category_name', 'thread_page_name', 'thread_page_num', 'thread_max_pages', 'thread_page_url']

columns_likes = ['num_likers', 'likers']
columns_quotes = ['num_quotes', 'quoted_post_ids', 'quoted_authors', 'quoted_contents']
columns_authors = ['author', 'author_title', 'author_num_posts', 'author_num_reviews', 'author_url', 'join_date_readable', 'join_date_data']

In [ ]:
def load_data(infile='nogit_data/list_of_post_contents.csv', nrows=None):
    # read, drop duplicates and NaNs in post_text, and convert post date string to a DateTime object

    df = pd.read_csv(infile, nrows=nrows)
    print(f'{df.columns=}')
    print(f'{df.shape=}')

    df.dropna(subset=['post_text'], inplace=True)
    print(f'{df.shape=}')
    df.drop_duplicates(subset=['post_text'], inplace=True)
    print(f'{df.shape=}')

    print(f'{columns_page_info=}')
    df = df[columns_page_info]
    df['posted_date_datetime'] = df.posted_date_readable.parallel_apply(
        lambda x: pd.to_datetime(x))
    return df

def get_discussions_only(df):
    # remove posts that come from reviews (vs. discussions)
    discussions = df[df.src_category_name.str.contains('Discussion')]
    # reformat 1,000 to 1000
    if discussions.author_num_posts.dtype != int:
        discussions.author_num_posts = discussions.author_num_posts.apply(lambda x: int(x.replace(',', '')))
    return discussions

# preprocess

In [ ]:
from time import sleep

In [ ]:
def nltk_preprocess(df, text_col_name='post_text', print_more=False):
    my_stopwords = stopwords.words('english')
    if print_more:
        print(my_stopwords)
    my_stopwords.extend([s.title() for s in my_stopwords])
    if print_more:
        print(f'{my_stopwords=}')
        print(f'{df.columns=}')

    porter = PorterStemmer()

    CUSTOM_FILTERS = [
        strip_tags, strip_punctuation, 
        lambda x: strip_short(x, minsize=2),  # remove only 1 letter words 
        lambda y: remove_stopwords(y, stopwords=my_stopwords),
        lambda z: porter.stem(z, to_lowercase=False )
    ]
    if print_more:
        print('pre-processing filters applied:', CUSTOM_FILTERS)

    df['preprocessed_posts'] = df[text_col_name].parallel_apply(
        lambda x: preprocess_string(x, CUSTOM_FILTERS)) 
    return df

def create_bigrams(df, text_col_name='preprocessed_posts', overwrite=False, outfile='word2vec.bigrams.model'):
    posts = df[text_col_name].to_list()
    my_phrases = gensim.models.Phrases(posts, min_count=1, threshold=10)
    bigram_ifier = Phraser(my_phrases)

    print('creating bigrams')
    #df['bigrammed_posts'] = df['preprocessed_posts'].parallel_apply(
    df['bigrammed_posts'] = df[text_col_name].progress_apply(
        lambda post: bigram_ifier[post]) 

    return df

def create_word2vec(df, bigram_col_name='bigrammed_posts', overwrite=False, outfile=f'word2vec.bigrams.gensim-{gensim.__version__}.model'):
    bigrammed_corpus = df[bigram_col_name].to_list()
    print(f'Creating word vectors for corpus size {len(bigrammed_corpus)=}, '
          f'example post {bigrammed_corpus[0]=}')
    print('creating model now!')
    model = Word2Vec(bigrammed_corpus, seed=42, workers=10)

    if overwrite: # TODO: test if this works with the sleep fxn
        # TODO WISHLIST: put sleep fxn into a decorator
        # TODO WISHLIST catch exception where parent folder doesn't exist; or 
        # force it to exist by committing to git (normally nogit_data ignored by git)
        # os.makedirs(os.path.dirname(outfile), exist_ok=True)
        print(f"\-- {overwrite=}! Saving model to {Path('nogit_data', 'Case_1', outfile)=}")
        print('Do you have a bkup? about to overwrite a file of size (TBD) created on ( )')
        sleep(1)
        print('2 sec to cancel')
        sleep(2)
        model.save(Path('nogit_data', 'Case_2', outfile))
    return model

In [ ]:
%%time

In [ ]:
%%time
#df = load_data(nrows=1000)
#df = load_data(nrows=10000)
#df = load_data(None)

# TODO: this makes assumptions that nogit_data is in the same folder
DATA_PATH = Path('nogit_data', 'Case_2')
def redo_or_load_forum(redo_steps=None, raw_csv_file=None, overwrite_results=None, sample_size=None, tag=''):
    # redo_steps, overwrite_results should be lists
    # 'clean_corpus', 'create_bigrams', 'create_word2vec' 
    # all filenames are relative to the data path (case_2 in nogit_data')
    # TODO: overwerite_results must be subset of redo_steps
    df, bigram_df, model = None, None, None
    if redo_steps: 
        print('redoing')
        print(f'{redo_steps=}')
        print(f'will overwrite {overwrite_results}')
        if 'load_data' in redo_steps: 
            '''
            # todo... there shouldn't be a separate load_data option
            if not raw_csv_file:
                raw_csv_file = Path('..', 'list_of_post_contents.csv')
            df = load_data(infile=Path(DATA_PATH, '..', raw_csv_file))#nrows=100000)
            if 'load_data' in ovewrite_results: 
                pass # not going to bother saving
            '''
            pass
        elif 'clean_corpus' in redo_steps:
            if not raw_csv_file:
                raw_csv_file = Path('..', 'list_of_post_contents.csv')
            df = load_data(infile=Path(DATA_PATH, raw_csv_file))
            df = nltk_preprocess(df)
            if 'clean_corpus' in overwrite_results:
                df.to_pickle(Path(DATA_PATH,f'{tag}preprocessed_df.pd-{pd.__version__}.pkl'))
        elif 'create_bigrams' in redo_steps:
            df = pd.read_pickle(Path(DATA_PATH,f'{tag}preprocessed_df.pd-1.5.1.pkl'))
            bigram_df = create_bigrams(df)
            if 'create_bigrams' in overwrite_results: 
                bigram_df.to_pickle(Path(DATA_PATH, f'{tag}bigram.df.pd-{pd.__version__}.pkl'))
        elif 'create_word2vec' in redo_steps:
            bigram_df = pd.read_pickle(Path(DATA_PATH, f'{tag}bigram.df.pd-1.5.1.pkl'))
            model = create_word2vec(bigram_df)
            print('created model')
            if 'create_word2vec' in overwrite_results:
                print('saving model')
                model.save(str(Path(DATA_PATH, f'{tag}bigram_word2vec.gensim-{gensim.__version__}.model')))
                print('saved')
    else:
        print('loading existing files')
        df = pd.read_pickle(Path(DATA_PATH,f'{tag}preprocessed_df.pd-1.5.1.pkl'))
        print('loaded preprocessed')
        bigram_df = pd.read_pickle(Path(DATA_PATH, f'{tag}bigram.df.pd-1.5.1.pkl'))
        print('loaded bigrams')
        model = Word2Vec.load(str(Path(DATA_PATH, f'{tag}bigram_word2vec.gensim-4.3.1.model')))
        print('loaded word2vec model')
    return df, bigram_df, model
# AttributeError: 'PosixPath' object has no attribute 'endswith'
# NOTE: CANNOT USE Path in model.save, on string !!!!

In [ ]:
pd.__version__
Path('test', f'bigram_word2vec.gensim-{gensim.__version__}.model')

In [ ]:
#print(model_mp.wv.most_similar('TS'))
#bigram_ifier = Phraser(my_phrases)

In [ ]:
%%time
#df_mp, bigram_mp, model_mp = redo_or_load_forum(redo_steps=['clean_corpus', 'create_bigrams', 'create_word2vec'], overwrite_results=['clean_cocreate_bigrams', 'create_word2vec'])
#df_mp, bigram_mp, model_mp = redo_or_load_forum(redo_steps=['create_word2vec'], overwrite_results=['create_word2vec'])
df_mp, bigram_mp, model_mp = redo_or_load_forum()

# query

In [ ]:
#model = Word2Ve./nogit_data/d('nogit_data/tmp_word2vec.model')

In [ ]:
tag = ''
model_mp = Word2Vec.load(str(Path(DATA_PATH, f'{tag}bigram_word2vec.gensim-4.3.1.model')))
my_model = model_mp
for query in ['SO']:#, 'MMS', 'HJ', 'HE', 'BJ', 'full_menu']:
  sims = my_model.wv.most_similar(query, topn=20)  # get other similar words
  #for item in sims:
  print(f'{query=}\t ' , ', '.join([ f'{word} = {vector:.2f}' for word, vector in sims]))
  print('-')

In [ ]:
def find_abbreviation(query, model=None, word_vectors=None):
    query = query.upper()
    if word_vectors:
        wv = word_vectors
    elif model:
        wv = model.wv
    else:
        return('Must provide model or word_vectors')
    try:
        similar_words = wv.most_similar(query, topn=50)  # get other similar words
    except KeyError as e:
        print(f'-- {query=}', e, '(Never encountered query in training corpus)')
        return
    phrases = [word for word, vector in similar_words if '_' in word]
    for phrase in phrases:
        bigram = phrase.split('_')
        candidate = ''.join([w[0] for w in bigram])
        if query == candidate.upper():
            print(f'{query=}, {phrase=}, \t Candidate: {phrase.replace("_", " ")}')
            return candidate
    print(f'-- {query=}, no expanded bigram found')
    unigrams = [word for word, vector in similar_words if '_' not in word]
    for word in unigrams:
        if word.upper() != query:
            if query[0] == word[0].upper():
                return print(f'{query=} \t Candidate UNIgram: {word}')
    print(f'-- {query=}, no expanded bigram NOR unigram found')


In [ ]:
find_abbreviation('TS', my_model)
find_abbreviation('ST', my_model)
find_abbreviation('BJ', my_model)
find_abbreviation('HE', my_model)
find_abbreviation('HJ', my_model)
find_abbreviation('FS', my_model)
find_abbreviation('SO', my_model)
find_abbreviation('CMT', my_model)


# print more detailed results, for acronyms that failed  


In [ ]:
a = my_model.wv.most_similar('10')
a
dict(a).keys()

In [ ]:
from funcy import silent
from collections import OrderedDict

def print_more(acronym, model=None, word_vectors=None, scores=False):
    if word_vectors:
        wv = word_vectors
    elif model:
        wv = model.wv
    else:
        return('Must provide model or word_vectors')

    similar_words = wv.most_similar(acronym, topn=100)  # get other similar words
    similar_words = dict(similar_words) 
    
    phrases = [word for word in similar_words.keys() if '_' in word]
    unigrams = [word for word in similar_words.keys() if '_' not in word]
    
    result = None
    for phrase in phrases:
        words = phrase.split('_')
        init_letters = [w[0] for w in words]
        candidate = ''.join(init_letters).upper()
        if acronym.upper() == candidate:
            result = candidate
            break
    if result:
        print(f'{acronym=}, Candidate: {phrase.replace("_", " ")}')
    else:
        print(f'{acronym=}, no candidate found')
        
    print()
    #print(f'{similar_words=}')
    print()
    if scores:
        display([f'{phrase}    {similar_words[phrase]:.2f}' for phrase in phrases[:5]])
        print()
        display([f'{word}     {similar_words[word]:.2f}' for word in unigrams[:5]])
    else:
        print(f'{acronym=}')
        display(phrases[:5])
        display(unigrams[:5])

In [ ]:
'TS'.split('_')

In [ ]:
print_more('ST', my_model, scores=True)

In [ ]:
print_more('CMT', my_model)

In [ ]:
find_abbreviation('RM', my_model)
find_abbreviation('CL', my_model)
find_abbreviation('TER', my_model)

find_abbreviation('CK', my_model) # not found in dictionary
find_abbreviation('CL', my_model) # not found in dictionary

find_abbreviation('LE', my_model)
find_abbreviation('CC', my_model)

find_abbreviation('HE', my_model)
find_abbreviation('FS', my_model)
find_abbreviation('HJ', my_model)
find_abbreviation('MP', my_model)
find_abbreviation('MMS',my_model)
find_abbreviation('SO', my_model) # not found in dictionary

find_abbreviation('MT', my_model)
# find_abbreviation('PV', my_model) # not found in dictionary
#print_more('TS', my_model) # not found in dictionary
find_abbreviation('SG', my_model) # not found in dictionary
find_abbreviation('SP', my_model) # not found in dictionary
find_abbreviation('TG', my_model) # not found in dictionary
find_abbreviation('TV', my_model) # not found in dictionary
find_abbreviation('WF', my_model) # not found in dictionary
find_abbreviation('WG', my_model) # not found in dictionary

find_abbreviation('FJ', my_model)
find_abbreviation('DP', my_model)
print_more('CL', my_model) # not found in dictionary # craiglist is one word
print_more('PS', my_model)
#print_more('BS', my_model)

In [ ]:
!rg "significant other" nogit_data/Case_2/KaggleForumMessages.csv 


# compared to pretreained

In [ ]:
'''
import gensim.downloader
# more details about the models are at https://github.com/RaRe-Technologies/gensim-data
#gnew_vectors = gensim.downloader.load('word2vec-google-news-300')
gnew_vectors = gensim.downloader.load('glove-twitter-25')
'''

In [ ]:
'''
find_abbreviation('lol', word_vectors=gnew_vectors)
print_more('sw', word_vectors=gnew_vectors)
# oh... they don't have bigrams in them ... 
# initialism
'''

# pretrained + fineturned on my dataset

In [ ]:
forum_posts = pd.read_csv(Path(DATA_PATH, 'KaggleForumMessages.csv'))
forum_posts['Message'] = forum_posts.Message.fillna('')
from bs4 import BeautifulSoup

In [ ]:
forum_posts[forum_posts.Message.isna()]

In [ ]:
forum_posts['post_text'] = forum_posts['Message'].progress_apply(lambda x: BeautifulSoup(x, "lxml").text)
# parallel apply may choke silently (make no progress) if underlying exception thrown
forum_posts

In [ ]:
forum_posts.to_pickle(Path(DATA_PATH,'kaggle_stripped_html'))

In [ ]:
if False: 
    kaggle_df = nltk_preprocess(forum_posts)
    kaggle_df.to_pickle(Path(DATA_PATH,'kaggle_nltk_processed.df.pkl'))
    bigram_kaggle = create_bigrams(kaggle_df)
    bigram_kaggle.to_pickle(Path(DATA_PATH,'kaggle_bigram.df.pkl'))
    model_kaggle = create_word2vec(bigram_kaggle)
    model_kaggle.save(str(Path(DATA_PATH,'kaggle.model')))
model_kaggle = Word2Vec.load(str(Path(DATA_PATH,'kaggle.model')))

In [ ]:
#wiki_model = api.load("glove-wiki-gigaword-50")
'''
redo_or_load(redo_steps=['clean_corpus', 'create_bigrams', 'create_word2vec'], 
             overwrite_results=['clean_corpus', 'create_bigrams', 'create_word2vec'],
             raw_csv_file='KaggleForumMessages.csv',
             tag='kaggle-'
            )
'''

In [ ]:
from funcy import silent
my_model = model_kaggle 
def tmp_wrapper():
    find_abbreviation('CC', my_model)
    find_abbreviation('HE', my_model)
    find_abbreviation('FS', my_model)
    find_abbreviation('LE', my_model)
    find_abbreviation('MP', my_model)
    find_abbreviation('MT', my_model)
    # find_abbreviation('PV', my_model) # not found in dictionary
    print_more('SO', my_model) # not found in dictionary
    find_abbreviation('SG', my_model) # not found in dictionary
    find_abbreviation('SO', my_model) # not found in dictionary
    find_abbreviation('SP', my_model) # not found in dictionary
    find_abbreviation('TV', my_model) # not found in dictionary
    find_abbreviation('WF', my_model) # not found in dictionary
    find_abbreviation('WG', my_model) # not found in dictionary
    find_abbreviation('HJ', my_model)

    find_abbreviation('FJ', my_model)
    find_abbreviation('DP', my_model)
    print_more('CL', my_model) # not found in dictionary # craiglist is one word
    print_more('PS', my_model)
    #print_more('BS', my_model)
    find_abbreviation('TG', my_model) # not found in dictionary
tmp_wrapper()

# for paper

In [ ]:
initialisms = ['FS', 'HE', 'LE', 'ST', 'TS', 'CL', 'RM', 'BP'] 
my_model = model_mp
for abbr in initialisms:
    print(find_abbreviation(abbr, my_model))
    print(print_more(abbr, my_model))